# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

# _Pre-Requisite: Make Sure You Created an Athena Table for Both TSV and Parquet in Previous Notebooks_

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
print(ingest_create_athena_table_tsv_passed)

True


In [3]:
%store -r ingest_create_athena_table_parquet_passed

In [4]:
print(ingest_create_athena_table_parquet_passed)

True


# Setup

In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [6]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

_dataset (bool)_ – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [7]:
p_filter = lambda x: x["product_category"] == "Digital_Software"

In [8]:
path = "s3://{}/amazon-reviews-pds/parquet/".format(bucket)
df_parquet_results = wr.s3.read_parquet(
    path, columns=["star_rating", "product_category", "review_body"], partition_filter=p_filter, dataset=True
)
df_parquet_results.shape

(102084, 3)

In [9]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,5,Covers all the important area's of threat. Kee...,Digital_Software
1,5,in all the years that I've used it I have neve...,Digital_Software
2,4,"could not e file, could not download data,yr t...",Digital_Software
3,5,I have used Avast for several years and it has...,Digital_Software
4,4,It had been awhile since I first started using...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [10]:
path = "s3://{}/amazon-reviews-pds/parquet/".format(bucket)
chunk_iter = wr.s3.read_parquet(
    path,
    columns=["star_rating", "product_category", "review_body"],
    # filters=[("product_category", "=", "Digital_Software")],
    partition_filter=p_filter,
    dataset=True,
    chunked=True,
)

In [11]:
print(next(chunk_iter))

       star_rating                                        review_body  \
0                5  Covers all the important area's of threat. Kee...   
1                5  in all the years that I've used it I have neve...   
2                4  could not e file, could not download data,yr t...   
3                5  I have used Avast for several years and it has...   
4                4  It had been awhile since I first started using...   
...            ...                                                ...   
16049            1  While the software in the box will work for Ma...   
16050            5  The Software Download Sample is a pretty slick...   
16051            1  The Amazon Downloader doesn't work for this pr...   
16052            5  I have used used Turbotax to do my taxes for 1...   
16053            3  My biggest complaint is that e-file costs extr...   

       product_category  
0      Digital_Software  
1      Digital_Software  
2      Digital_Software  
3      Digital_Soft

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [12]:
database_name = "dsoaws"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [13]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table["Name"])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [14]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet), database=database_name)

CPU times: user 669 ms, sys: 36.9 ms, total: 706 ms
Wall time: 6.79 s


In [15]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,45662660,R2424MK0YF60DH,BT00CTP84S,775486538,Amazon.com Gift Card in a Greeting Card (Vario...,5,0,0,N,Y,Easiest and most convenient experience ever!,I had three graduates and a bride to get gifts...,2012,2012-06-07,Gift Card
1,US,48337268,R2ZKW0XDFNKCIB,B004LLIKVU,473048287,Amazon.com eGift Cards,5,0,0,N,Y,Gift Cards Rock,What a great way to give a last minute gift. ...,2012,2012-06-07,Gift Card
2,US,40892729,R1WQ736I3JW7JV,BT00CTOUNS,775486538,Amazon.com Gift Card in a Greeting Card (Vario...,5,0,0,N,Y,mother's day gift card,I received this card from my son. It was easy...,2012,2012-06-07,Gift Card
3,US,46868983,R14CIF2YY8Y8AW,BT00CTOUNS,775486538,Amazon.com Gift Card in a Greeting Card (Vario...,4,0,0,N,N,Amazon Gift card,Took me a little while to figure how to use it...,2012,2012-06-07,Gift Card
4,US,46536369,RXGAM1GN9DBLR,BT00DDC7BK,802996579,Amazon.com Gift Cards - Print at Home,4,0,0,N,N,On-line gift card,The on-line gift card was quick and easy. My ...,2012,2012-06-07,Gift Card


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.  

_This will take a few seconds._

In [16]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet),
    database="{}".format(database_name),
    chunksize=64_000,  # 64 KB Chunks
)

CPU times: user 464 ms, sys: 26.6 ms, total: 491 ms
Wall time: 4.97 s


In [17]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    41754720  R19OFJV91M7D8X  B000YMR61A      141393130   
1             US    51669529  R1I6G894K5AGG5  B000YMR61A      141393130   
2             US    24731012  R17OE43FFEP81I  B000YMR5X4      234295632   
3             US    16049580  R15MGDDK63B52Z  B000YMR61A      141393130   
4             US    46098046  R1GGJJA2R68033  B000YMNI2Q      847631772   
...          ...         ...             ...         ...            ...   
3966          US    37257504  R2QF11SV16WQ9E  B009HBCU9W      194449368   
3967          US    28452005  R30JGXVFO63SZJ  B008S0IWNQ      710343641   
3968          US    52157231  R3DW1CXLWM1R5F  B00BFYHBY8      422702117   
3969          US    52201716  R1ESUSBZRTM05F  B008S0IR6I      249974728   
3970          US    38774886   RQBDBSC2758N6  B008S0IE5M      835509971   

                                      product_title  star_rating  \
0              TurboTax Deluxe 